In [1]:
from evalplus.data import (
    get_human_eval_plus,
    get_human_eval_plus_hash,
    get_mbpp_plus,
    get_mbpp_plus_hash,
    load_solutions,
)
import os
import json
from os.path import join
from tqdm import tqdm
work_dir = "/mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/mbpp"
#model_name = "deepseek-coder-6.7b-instruct_temp_1.2"
model_name = "code-llama-13b-instruct_temp_1.6"
debug_info = "_debug3_sd-ut"
gen_dir = model_name + debug_info + "_b4process"
original_dir = model_name + "_debug2_sd-ut"

with open(f"{work_dir}/{original_dir}/eval_results.json", "r") as f:
    eval_results = json.load(f)
for task_id in eval_results["eval"]:
    eval_results["eval"][task_id] = sorted(eval_results["eval"][task_id], key=lambda x: int(x["solution_id"]))
if work_dir.endswith("humaneval"):
    problems = get_human_eval_plus()
else:
    problems = get_mbpp_plus()



In [2]:
os.makedirs(join(work_dir, model_name + debug_info), exist_ok=True)
count_diff = 0
for sample in tqdm(load_solutions(join(work_dir, gen_dir))):
    task_id = sample["task_id"]
    p_name = task_id.replace("/", "_")
    os.makedirs(join(work_dir, model_name + debug_info, p_name), exist_ok=True)
    solution_id = int(sample["solution_id"])
    assert eval_results["eval"][task_id][solution_id]["solution_id"] == str(solution_id)
    assert "solution" in sample
    solution = (
        sample["solution"]
        if "solution" in sample
        else problems[task_id]["prompt"] + sample["completion"]
    )
    
    ### now we process the solution ###
    #assert "correct" in solution or "incorrect" in solution, f"{solution_id} {task_id} {solution}"
    if "### fixed code" in solution:
        count_diff += 1
        try:
            assert "```python\n" in solution, f"{solution_id} {task_id} {solution}"
            if "incorrect" not in solution:
                #print(solution_id, task_id)
                #print(solution)
                pass
            new_solution = solution.split("```python\n")[1].strip()
        except:
            new_solution = ""
        try:
            assert "```" not in new_solution
        except:
            #print(task_id, solution_id)
            #print(new_solution)
            new_solution = solution.split("```")[0].strip()
    else:
        try:
            assert "correct" in solution
        except:
            #print(solution_id, task_id)
            #print(solution)
            pass
        new_solution = eval_results["eval"][task_id][solution_id]["solution"]
    ### end of processing ###
    
    with open(join(work_dir, model_name + debug_info, p_name, f"{solution_id}.py"),"w",encoding="utf-8",) as f:
        f.write(new_solution)

print("Saved to", join(work_dir, model_name + debug_info))

79000it [00:11, 6699.28it/s]

Saved to /mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/mbpp/code-llama-13b-instruct_temp_1.6_debug3_sd-ut


In [16]:
continue_debug = {}
for sample in tqdm(load_solutions(join(work_dir, gen_dir))):
    task_id = sample["task_id"]
    if task_id not in continue_debug:
        continue_debug[task_id] = {}
    p_name = task_id.replace("/", "_")
    solution_id = int(sample["solution_id"])
    assert eval_results["eval"][task_id][solution_id]["solution_id"] == str(solution_id)
    assert "solution" in sample
    solution = (
        sample["solution"]
        if "solution" in sample
        else problems[task_id]["prompt"] + sample["completion"]
    )
    
    ### now we process the solution ###
    #assert "correct" in solution or "incorrect" in solution, f"{solution_id} {task_id} {solution}"
    if "### fixed code" in solution:
        continue_debug[task_id][solution_id] = True
    else:
        continue_debug[task_id][solution_id] = False
    ### end of processing ###
for task_id in continue_debug:
    #sort by solution_id
    continue_debug[task_id] = {str(k): v for k, v in sorted(continue_debug[task_id].items(), key=lambda x: int(x[0]))}

#save continue_debug to json
with open(join(work_dir, gen_dir.replace("_b4process", ""), "continue_debug.json"), "w") as f:
    print("Saved to", join(work_dir, gen_dir.replace("_b4process", ""), "continue_debug.json"))
    json.dump(continue_debug, f, indent=4)



32800it [00:01, 18122.06it/s]


Saved to /mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/humaneval/code-llama-7b-instruct_temp_1.6_debug2_sd-ut/continue_debug.json


In [21]:
from evalplus.data import (
    get_human_eval_plus,
    get_human_eval_plus_hash,
    get_mbpp_plus,
    get_mbpp_plus_hash,
    load_solutions,
)
import os
import json
from os.path import join
from tqdm import tqdm
work_dir = "/mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/humaneval"
#model_name = "deepseek-coder-7b-instruct-v1.5_temp_1.2"
model_name = "code-llama-7b-instruct_temp_1.6"
debug_info = "_debug1_sd-ut"
gen_dir = model_name + debug_info + "_b4process"
original_dir = model_name + "_debug1_sd-ut"
# read continue_debug.json
with open(join(work_dir, original_dir, "continue_debug.json"), "r") as f:
    continue_debug = json.load(f)

count_continues = 0
for task_id in continue_debug:
    count_continues += sum(continue_debug[task_id].values())
count_continues

20776

In [2]:
from pyminifier.minification import remove_comments_and_docstrings, remove_blank_lines

In [3]:
def clean_comment(code):
    code = remove_comments_and_docstrings(code)
    code = remove_blank_lines(code)
    return code